# Đồ án cuối kỳ

Thông tin nhóm:
    
Phạm Quang Phước Nguyên - MSSV: 1612441

Nguyễn Cao Nhân - MSSV: 1612451

---
## Đặt vấn đề

---
## Parse trang web để lấy dữ liệu

In [1]:
from requests_html import HTMLSession
import requests
import time # Dùng để sleep chương trình
import datetime as dt # Dùng để xử lý dữ liệu thời gian
import re

In [2]:
def get_estate_url_list(url):
    estate_info_urls = []
    while True:
        r = session.get(url)
        print(url)     

        estate_list = r.html.find(".RealEstate_listing", first=True)
        estate_items = estate_list.find(".RealEstate_item")    

        for estate_item in estate_items:
            tilte = estate_item.find(".link", first=True)
            item_url = tilte.attrs["href"]
            estate_info_urls.append(item_url)
            #print(item_url) 

        pages = r.html.find(".pagination", first=True).find("li")
        next_page = pages[-1].find("a", first=True)
        if next_page:
            url = next_page.attrs["href"]
        else:
            break 

        time.sleep(0.5)
    return estate_info_urls

In [3]:
prop_list = ['address', 'price', 'size', 'exprire_data', 'update_date', 'object', 'sex', 'amenities', 'electric_price', 'water_price', 'internet_price', 'payment_method', 'close_time']#, 'detail']

prop_dict = {'Giới tính':'sex', 'Giá điện':'electric_price', 'Giá nước':'water_price', 'Giá wifi':'internet_price', 'Hình thức thanh toán':'payment_method', 'Giờ đóng cửa':'close_time'}

In [4]:
def get_estate_info(estate_info_url):
    r = session.get(estate_info_url).html
    
    info = {}
    info['url'] = estate_info_url

    info['address'] = r.find(".pr_location", first=True).find(".address", first=True).text

    pr_price_acreage = r.find(".pr_price_acreage", first=True)
    info['price'] = pr_price_acreage.find(".price", first=True).text
    size = pr_price_acreage.find(".acreage", first=True).text
    info['size'] = re.findall(r"\d+", size)[0]

    update_at = pr_price_acreage.find(".update_at")
    for date in update_at:
        text = date.text
        if text.startswith("Cập nhật"):
            info['update_date'] = re.findall(r"\d{2}/\d{2}/\d{4}", text)[0]
        elif text.startswith("Ngày hết hạn"):
            info['exprire_data'] = re.findall(r"\d{2}/\d{2}/\d{4}", text)[0]  

    highlight = r.find(".RealEstate_Highlight", first=True).find(".item")
    for item in highlight:
        name = item.find(".name", first=True).text
        value = item.find(".value", first=True).text
        prop_name = prop_dict.get(name)
        if prop_name:
            info[prop_name] = value
        elif name == 'Đối tượng':
            info['object'] = value.split(", ")

    amenities = []
    amenities_list = r.find(".pr_amenities_list", first=True)    
    if amenities_list:
        amenities_list = amenities_list.find(".pr_amenitie_item")
        for amenity in amenities_list:
            amenities.append(amenity.text)
    info['amenities'] = amenities

    info['detail'] = r.find(".RealEstate_Content", first=True).find(".content", first=True).text

    return info        

In [5]:
session = HTMLSession()
url = "https://bds123.vn/cho-thue-phong-tro-nha-tro-quan-5-ho-chi-minh.html"

estate_info_urls = get_estate_url_list(url)

https://bds123.vn/cho-thue-phong-tro-nha-tro-quan-5-ho-chi-minh.html
https://bds123.vn/cho-thue-phong-tro-nha-tro-quan-5-ho-chi-minh.html?page=2
https://bds123.vn/cho-thue-phong-tro-nha-tro-quan-5-ho-chi-minh.html?page=3
https://bds123.vn/cho-thue-phong-tro-nha-tro-quan-5-ho-chi-minh.html?page=4
https://bds123.vn/cho-thue-phong-tro-nha-tro-quan-5-ho-chi-minh.html?page=5
https://bds123.vn/cho-thue-phong-tro-nha-tro-quan-5-ho-chi-minh.html?page=6
https://bds123.vn/cho-thue-phong-tro-nha-tro-quan-5-ho-chi-minh.html?page=7
https://bds123.vn/cho-thue-phong-tro-nha-tro-quan-5-ho-chi-minh.html?page=8


In [6]:
#open dataset file
dataset_file = open("nhatro.csv", "w", encoding='utf-8')

#write header
dataset_file.write("url")
for prop in prop_list:
    dataset_file.write("\t " + prop)
dataset_file.write("\n")

#write records
print("Start parsing {} HTML pages".format(len(estate_info_urls)))
k = 0
current_percent = 0

for estate_info_url in estate_info_urls:    
    estate_info = get_estate_info(estate_info_url)
    
    dataset_file.write(estate_info['url'])    
    for prop in prop_list:
        cell = estate_info.get(prop, "")
        dataset_file.write("\t " + str(cell))
    dataset_file.write("\n")
    
    k+=1
    percent = int(k * 100/ len(estate_info_urls))
    if percent > current_percent and percent % 5 == 0:
        current_percent = percent
        print("Finished {} % ...".format(percent))
    
#save dataset
dataset_file.close()

print("Dataset Writing Finish")

Start parsing 150 HTML pages
Finished 5 % ...
Finished 10 % ...
Finished 15 % ...
Finished 20 % ...
Finished 25 % ...
Finished 30 % ...
Finished 35 % ...
Finished 40 % ...
Finished 45 % ...
Finished 50 % ...
Finished 55 % ...
Finished 60 % ...
Finished 65 % ...
Finished 70 % ...
Finished 75 % ...
Finished 80 % ...
Finished 85 % ...
Finished 90 % ...
Finished 95 % ...
Finished 100 % ...
Dataset Writing Finish


---
## Mô tả dữ liệu

---
## Import

In [7]:
#khai báo thư viện
import numpy as np
%matplotlib inline
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

---
## Đọc dữ liệu từ file csv vào dataframe

---
## Trực quan hóa dữ liệu

---
## Tiền xử lí dữ liệu

---
## Train mô hình

---
## Test mô hình

---
## Đánh giá kết quả

---
## Kết luận